<a href="https://colab.research.google.com/github/adudars/PedidON/blob/master/arquivos/9.10_Subconsultas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9.10 SUBCONSULTAS

## ESTABELECENDO CONEXÃO




### Instalando bibliotecas de acesso

In [1]:
#!pip install --user psycopg2-binary
#!pip install --user pandas

### Importando bibliotecas

In [3]:
import pandas as pd
import psycopg2

### Conectando com o banco através do psycopg2

In [4]:
conn = psycopg2.connect(host="lallah.db.elephantsql.com", database="nxthbnlu", user="nxthbnlu", password="buyAYKsY17pqexg1grIcJaWm0gHBsO2F")

## SUBCONSULTAS

### Categorias que possuem itens associados 

In [5]:
categoria = pd.read_sql_query("""select nome_categoria
                                  from categoria
                                  where id_categoria in (select fk_id_categoria from item_cardapio)""", conn)

In [6]:
categoria

,nome_categoria
0,entrada
1,drink
2,principal
3,sobremesa


### Pedidos associados à observação

In [10]:
pedidos = pd.read_sql_query("""select *
                              from pedido
                              where id_pedido in (select fk_id_pedido from observacao)""", conn)

In [11]:
pedidos

,id_pedido,quantidade,hora,fk_id_comanda,fk_id_itemcardapio
0,1,2,12:58:00,1,1
1,4,3,12:45:00,3,4
2,5,1,12:45:00,3,5
3,7,1,12:23:00,4,7
4,8,1,12:37:00,4,8
5,9,2,13:42:00,5,9
6,10,1,12:26:00,6,10
7,11,1,12:57:00,6,1
8,12,2,12:21:00,7,2
9,14,3,13:38:00,8,4


### Funcionários associados às comandas

In [14]:
qtd_atendimentos = pd.read_sql_query("""select nome_funcionario, count(cd.id_comanda) as quantidade_atendimentos
                                      from funcionario as f
                                      inner join comanda as cd on
                                      (f.id_funcionario = cd.fk_id_funcionario)
                                      where id_comanda in (select fk_id_comanda from pedido)
                                      group by nome_funcionario""", conn)

In [15]:
qtd_atendimentos

,nome_funcionario,quantidade_atendimentos
0,Matheus,5
1,Bernardo,4
2,Ismael,4


### Itens da comanda por cliente

In [16]:
itens_cliente = pd.read_sql_query("""select fk_id_comanda as comanda, nome_cliente, nome_item
                                          from pedido as p
                                          inner join comanda as cd on
                                          (cd.id_comanda = p.fk_id_comanda)
                                          inner join cliente as c on
                                          (cd.fk_id_cliente = c.id_cliente)
                                          inner join item_cardapio as ic on
                                          (p.fk_id_itemcardapio = ic.id_itemcardapio)
                                          where id_pedido in (select id_pedido
                                                        			from item_cardapio as ic
                                                              inner join pedido as p on
                                                              (ic.id_itemcardapio = p.fk_id_itemcardapio))
                                          order by comanda""", conn)

In [17]:
itens_cliente

,comanda,nome_cliente,nome_item
0,1,Lara,Reta da Penha
1,1,Lara,Rua Sete
2,2,Jorge,Parque Moscoso
3,3,Clara,Convento da Penha
4,3,Clara,Ilha de Socó
5,3,Clara,Bons ventos te trazem
6,4,Davi,Barlavento
7,4,Davi,Stella Artois
8,5,Milton,Pepsi
9,6,Jorge,Cidade Sol
